In [13]:
import numpy as np 
import pandas as pd
import requests # for HTTP requests
import xlsxwriter 
import math

In [14]:
pip install alpha_vantage

     |████████████████████████████████| 647 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 7.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [45]:
# import additional packages
import alpha_vantage
from alpha_vantage.timeseries import TimeSeries

In [63]:
# take a look at the stocks
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [20]:
ALPHA_VANTAGE_API_KEY = 'ZS77JTEY88MPC7SD'

In [24]:
ts = TimeSeries(key = ALPHA_VANTAGE_API_KEY,output_format='pandas')

In [53]:
ticker = 'AAPL'
data = ts.get_monthly_adjusted(ticker)
# data[0].iloc[0]
# get the low for each day
data[0].iloc[0]

1. open               1.337500e+02
2. high               1.378770e+02
3. low                1.309300e+02
4. close              1.353900e+02
5. adjusted close     1.353900e+02
6. volume             6.581963e+08
7. dividend amount    2.050000e-01
Name: 2021-02-10 00:00:00, dtype: float64

In [51]:
from alpha_vantage.fundamentaldata import FundamentalData
fd = FundamentalData(ALPHA_VANTAGE_API_KEY,output_format = 'pandas')
data = fd.get_income_statement_quarterly('AAPL')
data[0].T

date,1970-01-01 00:00:00.000000000,1970-01-01 00:00:00.000000001,1970-01-01 00:00:00.000000002,1970-01-01 00:00:00.000000003,1970-01-01 00:00:00.000000004,1970-01-01 00:00:00.000000005,1970-01-01 00:00:00.000000006,1970-01-01 00:00:00.000000007,1970-01-01 00:00:00.000000008,1970-01-01 00:00:00.000000009,...,1970-01-01 00:00:00.000000011,1970-01-01 00:00:00.000000012,1970-01-01 00:00:00.000000013,1970-01-01 00:00:00.000000014,1970-01-01 00:00:00.000000015,1970-01-01 00:00:00.000000016,1970-01-01 00:00:00.000000017,1970-01-01 00:00:00.000000018,1970-01-01 00:00:00.000000019,1970-01-01 00:00:00.000000020
fiscalDateEnding,2020-12-31,2020-09-30,2020-06-30,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,...,2018-03-31,2017-12-31,2017-09-30,2017-06-30,2017-03-31,2016-12-31,2016-09-30,2016-06-30,2016-03-31,2015-12-31
reportedCurrency,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD,...,USD,USD,USD,USD,USD,USD,USD,USD,USD,USD
totalRevenue,111439000000,64698000000,59685000000,58313000000,84310000000,64040000000,53809000000,58015000000,88293000000,62900000000,...,61137000000,52579000000,52579000000,45408000000,52896000000,78351000000,46852000000,42358000000,50557000000,75872000000
totalOperatingExpense,10794000000,9914000000,9589000000,9517000000,8685000000,8688000000,8683000000,8406000000,7638000000,7966000000,...,7528000000,6811000000,6811000000,6720000000,6494000000,6817000000,6052000000,6001000000,5934000000,6252000000
costOfRevenue,67111000000,40009000000,37005000000,35943000000,52279000000,39727000000,33582000000,36194000000,54381000000,38816000000,...,37715000000,32648000000,32648000000,27920000000,32305000000,48175000000,29039000000,26252000000,30636000000,45449000000
grossProfit,44328000000,24689000000,22680000000,22370000000,32031000000,24313000000,20227000000,21821000000,33912000000,24084000000,...,23422000000,19931000000,19931000000,17488000000,20591000000,30176000000,17813000000,16106000000,19921000000,30423000000
ebit,33579000000,18237000000,16586000000,16678000000,28191000000,20116000000,15710000000,17843000000,30509000000,20043000000,...,19699000000,17067000000,17067000000,14264000000,17546000000,24180000000,12188000000,10469000000,14142000000,24573000000
netIncome,28755000000,12673000000,11253000000,11249000000,22236000000,13686000000,10044000000,11561000000,19965000000,14125000000,...,13822000000,20065000000,10714000000,8717000000,11029000000,17891000000,9014000000,7796000000,10516000000,18361000000
researchAndDevelopment,5163000000,4978000000,4758000000,4565000000,4451000000,4110000000,4257000000,3948000000,3902000000,3750000000,...,3378000000,3407000000,2997000000,2937000000,2776000000,2871000000,2570000000,2560000000,2511000000,2404000000
effectOfAccountingCharges,None,None,None,None,None,None,None,None,None,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
my_columns = ['Ticker','Stock Price','Total Volume','Number Of Shares To Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Total Volume,Number Of Shares To Buy


In [65]:
symbol = ticker
price = data[0].iloc[0][2]
volume = data[0].iloc[0][5]

In [62]:
final_dataframe.append(
    # create series that lists all in a data frame. tell where to add the info
    pd.Series([
        symbol,
        price,
        volume,
        'N/A'
    ], index=my_columns),
    ignore_index=True
)

,Ticker,Stock Price,Total Volume,Number Of Shares To Buy
0,AAPL,130.93,658196261.0,N/A


In [84]:
import time
final_dataframe = pd.DataFrame(columns = my_columns)
# loop thru all stocks. ideally, we loop through all stocks in S&P 500, but we can't because of the free version of Alpha Vantage we have
# using time.sleep(15) to get around the 5 calls/minute restriction. For the sake of demonstration, we'll restrict ourselves to just 10 stocks
for stock in stocks['Ticker'][:10]:
    data = ts.get_monthly_adjusted(stock)
    low_price = data[0].iloc[0][2]
    volume = data[0].iloc[0][5]
    final_dataframe = final_dataframe.append(
        pd.Series([
            stock,
            low_price,
            volume,
            'N/A'
        ], index=my_columns),
        ignore_index=True
    )
    time.sleep(15)

In [85]:
final_dataframe

,Ticker,Stock Price,Total Volume,Number Of Shares To Buy
0,A,120.180,12483838.0,N/A
1,AAL,16.330,320302329.0,N/A
2,AAP,150.031,8830126.0,N/A
3,AAPL,130.930,658196261.0,N/A
4,ABBV,101.860,68462350.0,N/A
5,ABC,103.400,11180237.0,N/A
6,ABMD,321.010,2761121.0,N/A
7,ABT,118.790,41764997.0,N/A
8,ACN,243.970,13714304.0,N/A
9,ADBE,459.820,16419467.0,N/A


In [86]:
portfolio_size = input('Enter the value of your portfolio: ')

try: 
    val = float(portfolio_size)
except ValueError: # only respond to value errors
    Print('Please enter a valid number (not a string) /n')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio:  10000000


In [88]:
position_size = val/len(final_dataframe.index) # investing equally in all stocks, just for ease
print(position_size)

1000000.0


In [114]:
# number of shares of each to buy
for index in range(0,len(final_dataframe.index)):
    final_dataframe.loc[index, 'Number Of Stocks To Buy'] = math.floor(position_size / final_dataframe.loc[index, 'Stock Price']) # access final column

In [112]:
final_dataframe.drop(final_dataframe.columns[3],axis=1,inplace=True)
# already done. error arose because for some reason I ran this again!

In [115]:
final_dataframe

,Ticker,Stock Price,Total Volume,Number Of Stocks To Buy
0,A,120.180,12483838.0,8320.0
1,AAL,16.330,320302329.0,61236.0
2,AAP,150.031,8830126.0,6665.0
3,AAPL,130.930,658196261.0,7637.0
4,ABBV,101.860,68462350.0,9817.0
5,ABC,103.400,11180237.0,9671.0
6,ABMD,321.010,2761121.0,3115.0
7,ABT,118.790,41764997.0,8418.0
8,ACN,243.970,13714304.0,4098.0
9,ADBE,459.820,16419467.0,2174.0


In [129]:
# store final_dataframe in excel file
writer = pd.ExcelWriter('updated trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Updated Trades',index=False)

In [130]:
# define a color scheme
background_color = 'yellow'
font_color = 'blue'

In [131]:
# formatting
string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'num_format': '$0.00'
    }
)

integer_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'num_format':'0'
    }
)

In [134]:
# apply formatting to our xlsx file. We can do this programmatically using loops....but at this point I feel a tad lazy. Since this is a solo project, I'll just stick to this :P
writer.sheets['Updated Trades'].set_column('A:A',20,string_format)
writer.sheets['Updated Trades'].set_column('B:B',20,dollar_format)
writer.sheets['Updated Trades'].set_column('C:C',20,integer_format)
writer.sheets['Updated Trades'].set_column('D:D',20,integer_format)

# apply appropriate formatting to the column titles
writer.sheets['Updated Trades'].write('A1','Ticker',string_format)
writer.sheets['Updated Trades'].write('B1','Stock Price',dollar_format)
writer.sheets['Updated Trades'].write('C1','Total Volume',integer_format)
writer.sheets['Updated Trades'].write('D1','Number Of Shares To Buy',integer_format)

# simply save our file! Done.
writer.save()